## Python wrapper

In [1]:
from vowpalwabbit import pyvw

In [2]:
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
import os
import re
import numpy as np
from sklearn.metrics import accuracy_score

In [3]:
# поменяйте путь к файлу
PATH_TO_DATA = 'c:/Users\Виталий\Documents\Python Scripts\mlcourse.ai-master\data\imdb_reviews'
reviews_train = load_files(os.path.join(PATH_TO_DATA, 'train'))
text_train, y_train = reviews_train.data, reviews_train.target

In [4]:
reviews_test = load_files(os.path.join(PATH_TO_DATA, 'test'))
text_test, y_test = reviews_test.data, reviews_test.target

In [5]:
print("Number of documents in training data: %d" % len(text_train))
print(np.bincount(y_train))
print("Number of documents in test data: %d" % len(text_test))
print(np.bincount(y_test))

Number of documents in training data: 25000
[12500 12500]
Number of documents in test data: 25000
[12500 12500]


In [6]:
text_train[0]

b"Zero Day leads you to think, even re-think why two boys/young men would do what they did - commit mutual suicide via slaughtering their classmates. It captures what must be beyond a bizarre mode of being for two humans who have decided to withdraw from common civility in order to define their own/mutual world via coupled destruction.<br /><br />It is not a perfect movie but given what money/time the filmmaker and actors had - it is a remarkable product. In terms of explaining the motives and actions of the two young suicide/murderers it is better than 'Elephant' - in terms of being a film that gets under our 'rationalistic' skin it is a far, far better film than almost anything you are likely to see. <br /><br />Flawed but honest with a terrible honesty."

In [7]:
y_train[0] # хороший отзыв

1

In [8]:
text_train[1]

b'Words can\'t describe how bad this movie is. I can\'t explain it by writing only. You have too see it for yourself to get at grip of how horrible a movie really can be. Not that I recommend you to do that. There are so many clich\xc3\xa9s, mistakes (and all other negative things you can imagine) here that will just make you cry. To start with the technical first, there are a LOT of mistakes regarding the airplane. I won\'t list them here, but just mention the coloring of the plane. They didn\'t even manage to show an airliner in the colors of a fictional airline, but instead used a 747 painted in the original Boeing livery. Very bad. The plot is stupid and has been done many times before, only much, much better. There are so many ridiculous moments here that i lost count of it really early. Also, I was on the bad guys\' side all the time in the movie, because the good guys were so stupid. "Executive Decision" should without a doubt be you\'re choice over this one, even the "Turbulenc

In [9]:
y_train[1] # плохой отзыв

0

In [32]:
def to_vw_format(document, label=None):
    return str(label or '') + ' |text ' + ' '.join(re.findall('\w{3,}', document.lower())).replace(chr(10), '').replace(chr(13), '').replace(chr(9), '').replace(chr(11), '').replace(chr(12), '') + '\n'

In [11]:
ord('\f')

12

In [33]:
to_vw_format(str(text_train[0]), 1 if y_train[0] == 1 else -1)

'1 |text zero day leads you think even think why two boys young men would what they did commit mutual suicide via slaughtering their classmates captures what must beyond bizarre mode being for two humans who have decided withdraw from common civility order define their own mutual world via coupled destruction not perfect movie but given what money time the filmmaker and actors had remarkable product terms explaining the motives and actions the two young suicide murderers better than elephant terms being film that gets under our rationalistic skin far far better film than almost anything you are likely see flawed but honest with terrible honesty\n'

In [34]:
train_share = int(0.7 * len(text_train))
train, valid = text_train[:train_share], text_train[train_share:]
train_labels, valid_labels = y_train[:train_share], y_train[train_share:]

In [35]:
len(train_labels), len(valid_labels)

(17500, 7500)

In [36]:
with open('../../data/imdb_reviews/movie_reviews_train.vw', 'w') as vw_train_data:
    for text, target in zip(train, train_labels):
        vw_train_data.write(to_vw_format(str(text), 1 if target == 1 else -1))
with open('../../data/imdb_reviews/movie_reviews_valid.vw', 'w') as vw_train_data:
    for text, target in zip(valid, valid_labels):
        vw_train_data.write(to_vw_format(str(text), 1 if target == 1 else -1))
with open('../../data/imdb_reviews/movie_reviews_test.vw', 'w') as vw_test_data:
    for text in text_test:
        vw_test_data.write(to_vw_format(str(text)))

In [37]:
#vw_model = pyvw.vw(b=18, oaa=10, quiet=True)
vw_model = pyvw.vw(loss_function='hinge', ngram=2, quiet=True)

In [23]:
train_examples = [
  "-1 | price:.23 sqft:.25 age:.05 2006",
  "1 | price:.18 sqft:.15 age:.35 1976",
  "-1 | price:.53 sqft:.32 age:.87 1924",
]

for example in train_examples:
    vw_model.learn(example)

In [24]:
test_example = "| price:.46 sqft:.4 age:.10 1924"

prediction = vw_model.predict(test_example)
print(prediction)

-0.9841381311416626


In [18]:
from tqdm.notebook import tqdm

In [20]:
PATH_TO_DATA + '/movie_reviews_train1.vw'

'c:/Users\\Виталий\\Documents\\Python Scripts\\mlcourse.ai-master\\data\\imdb_reviews/movie_reviews_train1.vw'

In [38]:
with open(PATH_TO_DATA + '/movie_reviews_train.vw') as f:
    # '1 | question text'
    #for line in tqdm(f, total=17500):
    for line in f:
        print(line)
        vw_model.learn(line)

1 |text zero day leads you think even think why two boys young men would what they did commit mutual suicide via slaughtering their classmates captures what must beyond bizarre mode being for two humans who have decided withdraw from common civility order define their own mutual world via coupled destruction not perfect movie but given what money time the filmmaker and actors had remarkable product terms explaining the motives and actions the two young suicide murderers better than elephant terms being film that gets under our rationalistic skin far far better film than almost anything you are likely see flawed but honest with terrible honesty



TypeError: expecting single line example, got multi_ex of len 2

In [39]:
valid_predictions = []
with open(PATH_TO_DATA + '/movie_reviews_valid.vw') as f:
    # '1 | question text'
    for line in tqdm_notebook(f, total=7500):
        curr_prediction = vw_model.predict(line)
        valid_predictions.append(curr_prediction)

NameError: name 'tqdm_notebook' is not defined

In [ ]:
accuracy_score(valid_labels, valid_predictions)